In [ ]:
# CS 8803 Efficient Machine Learning
# [First Name Last Name]
# GT ID: [GT ID here]
# Date: [Date here]
#
# Lab 02: Quantization
#
# This lab will guide you through the fundamental concepts of model quantization,
# including uniform, non-uniform, and quantization-aware training (QAT).
# You will implement these techniques from scratch and apply them to a
# pretrained OPT-125m language model.

# Lab Overview and Instructions

In this lab, we will explore quantization with the OPT-125m model and WikiText-2 dataset to understand the basics of quantization and its implications on model performance and efficiency.

### Grading
The lab is divided into six main tasks with the following point distribution:
-   **Task 1: Setup and Baseline (15%)**
-   **Task 2: Uniform Quantization Implementation (20%)**
-   **Task 3: Non-Uniform (Power-of-Two) Quantization (10%)**
-   **Task 4: Applying and Analyzing Quantization (20%)**
-   **Task 5: Quantization-Aware Training (QAT) (20%)**
-   **Task 6: Advanced Quantization (15%)**

### Deliverables
You are required to submit this completed Jupyter Notebook. Ensure that you have:
1.  Filled in all the `#TODO` sections with functional code.
2.  Generated all the required plots and dataframes.
3.  Written your answers and analysis in the designated markdown cells.


# Task 1: Setup and Baseline [15%]

### Part 1a: Download the Dataset and Model [2.5%]

**Task:** Your first step is to set up the environment by loading the necessary model and dataset. You will use the Hugging Face `transformers` library to download the `facebook/opt-125m` model and the `datasets` library to load the `wikitext` dataset.

-   **Model:** `facebook/opt-125m` is a small, decoder-only transformer model, suitable for experimentation without requiring heavy GPU usage.
-   **Dataset:** `wikitext-2-raw-v1` is a standard language modeling benchmark dataset. We will use the 'train' and 'test' splits.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from copy import deepcopy
import torch.nn.functional as F
import gc
from dataclasses import dataclass
from torch.autograd import Function

# --- 1. Model and Tokenizer Setup ---
model_name = "facebook/opt-125m"

# TODO: Load the tokenizer for the specified model.

tokenizer = # TODO

# OPT models don't have a default padding token, so we set it to the EOS (End Of Sentence) token.
# This is important for batching inputs of different lengths.
tokenizer.pad_token = tokenizer.eos_token

device = "cuda" if torch.cuda.is_available() else "cpu"

# TODO: Load the pre-trained causal language model.
# Ensure the model is moved to the correct device
model = # TODO

# --- 2. Load the dataset ---
# We'll use the 'wikitext-2-raw-v1' version of the wikitext dataset.

# TODO: Load the 'test' split for final evaluation.
test_dataset = # TODO

# TODO: Load the 'train' split, which we will use for QAT.
train_dataset = # TODO

In [ ]:
# DO NOT MODIFY THIS CELL

# This cell contains helper functions to tokenize and prepare the data for evaluation.

def tokenize_and_chunk(texts, tokenizer, seq_length=512):
    """Tokenizes text and splits it into fixed-length chunks."""
    all_tokens = tokenizer(
        "\n\n".join(texts),
        return_tensors="pt",
        truncation=False,
    ).input_ids.squeeze()
    data_chunks = []
    for i in range(0, all_tokens.size(0) - seq_length, seq_length):
        data_chunks.append(all_tokens[i : i + seq_length])
    return data_chunks

class PerplexityDataset(torch.utils.data.Dataset):
    """A torch Dataset for perplexity evaluation."""
    def __init__(self, chunks):
        self.chunks = chunks
    def __len__(self):
        return len(self.chunks)
    def __getitem__(self, idx):
        return self.chunks[idx]

# Create dataloaders for evaluation and training
eval_data_chunks = tokenize_and_chunk(test_dataset["text"], tokenizer, seq_length=512)
eval_dataset = PerplexityDataset(eval_data_chunks)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

train_subset_size = 512
train_data_chunks = tokenize_and_chunk(train_dataset["text"][: (train_subset_size * 2)], tokenizer, seq_length=512)
train_data_chunks = train_data_chunks[:train_subset_size]
train_p_dataset = PerplexityDataset(train_data_chunks)
train_dataloader = DataLoader(train_p_dataset, batch_size=8, shuffle=True)

### Part 1b: Establish Full-Precision Baseline [2.5%]

**Task:** Evaluate the original, full-precision (FP32) model to establish a baseline. We will use **perplexity** as our primary metric. Answer the following question for credit.


**Question:** What is perplexity?

**[WRITE YOUR ANSWER HERE]**

In [ ]:
# Run this cell to evaluate the full precision model
# DO NOT MODIFY THIS CELL
# This function for evaluating perplexity is provided for you.

def evaluate_perplexity(model, dataloader, device):
    model.eval()
    total_nll = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Perplexity"):
            input_ids = batch.to(device)
            num_tokens_in_batch = input_ids.numel()

            outputs = model(input_ids, labels=input_ids)

            total_nll += outputs.loss.item() * num_tokens_in_batch
            total_tokens += num_tokens_in_batch

    avg_nll = total_nll / total_tokens
    perplexity = torch.exp(torch.tensor(avg_nll))

    return avg_nll, perplexity.item()


# --- Calculate baseline perplexity ---
loss_fp, ppl_fp = evaluate_perplexity(model, eval_dataloader, device)
print(f"Full precision model: loss {loss_fp:.3f}, perplexity {ppl_fp:.3f}")

### Part 1c: Analyze Model Size and Weight Distribution [10%]

**Task:** Before quantizing, it's important to understand the model's structure and size. You will implement helper functions to count model parameters and estimate the memory footprint after quantization.

In [ ]:
# Print the model architecture to understand its layers.
print(model)

In [ ]:
# --- Helper functions to analyze model size ---

def count_params(module: nn.Module):
    """
    Counts the total number of parameters in a given PyTorch module.

    Input:
        module (nn.Module): The module to analyze.
    Output:
        int: The total number of parameters.
    """
    # TODO: Calculate the total number of parameters.
    # Hint: Iterate through module.parameters() and sum the .numel() of each parameter tensor.
    return # TODO

def estimated_weight_size_bytes(m: nn.Module, w_bits=8):
    """
    Estimates the memory size of a model's weights after quantization.
    This function assumes biases are kept in 32-bit floating point.

    Inputs:
        m (nn.Module): The model.
        w_bits (int): The number of bits for quantized weights.
    Output:
        int: The estimated total size of the weights in bytes.
    """
    total_bytes = 0
    # TODO: Calculate the estimated size in bytes
    # Iterate through all modules in the model using m.modules().
    # For each nn.Linear layer, calculate the quantized weight size and add the bias size.
    # Hint: bias is kept in fp32
    return # TODO


# Get the first OPT decoder layer parameters
first_decoder_layer = model.model.decoder.layers[0]
# TODO: Check the first decoder layer structure, and for each Linear layer:
# (1) Print the linear layer name
# (2) Print the total number of parameters
# (3) Print the estimated weight size in bytes



#### Visualize Weight Ranges and Granularity

**Task:** Visualize the weight ranges (minimum and maximum values) for the `fc1` layer in the first decoder block of the model. This analysis helps understand why finer-grained quantization can reduce error. You will compare three different **quantization granularities**:

1.  **Per-Tensor:** One min/max value for the entire weight matrix.
2.  **Per-Channel:** One min/max value for each output channel (i.e., each row).
3.  **Per-Group:** One min/max value for small groups of weights within each channel.

In [ ]:
# --- Visualize the weight ranges (min/max) for the fc1 Linear layer ---

# 1. Get the weight tensor from the first fc1 layer
fc1_layer = model.model.decoder.layers[0].fc1
W = fc1_layer.weight.detach().cpu()

# TODO: Calculate per-tensor min/max
per_tensor_min = # TODO
per_tensor_max = # TODO

# TODO: Calculate per-channel min/max
# For a linear layer weight (out_features, in_features), the channel axis is 0.
per_channel_min = # TODO
per_channel_max = # TODO

# TODO: Calculate per-group min/max
group_size = 32
W_grouped = # TODO
per_group_min = # TODO
per_group_max = # TODO

# --- Plotting --- (This code is provided for you)
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=False)
fig.suptitle("Weight Range Visualization for Different Granularities (fc1 layer)", fontsize=16)

axs[0].plot(per_channel_min.numpy(), label='Per-Channel Min', color='blue', alpha=0.7)
axs[0].plot(per_channel_max.numpy(), label='Per-Channel Max', color='red', alpha=0.7)
axs[0].axhline(per_tensor_min, ls='--', color='blue', label='Per-Tensor Min')
axs[0].axhline(per_tensor_max, ls='--', color='red', label='Per-Tensor Max')
axs[0].set_title("Per-Channel vs. Per-Tensor Granularity")
axs[0].set_xlabel("Output Channel Index")
axs[0].set_ylabel("Weight Value")
axs[0].legend()
axs[0].grid(True)

axs[1].plot(per_group_min.numpy(), label='Per-Group Min', color='green')
axs[1].plot(per_group_max.numpy(), label='Per-Group Max', color='orange')
axs[1].set_title(f"Per-Group Granularity (Group Size = {group_size})")
axs[1].set_xlabel("Group Index")
axs[1].set_ylabel("Weight Value")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**Question:** Based on the visualization, explain how finer granularity (like per-channel or per-group) can reduce quantization error compared to per-tensor quantization.

**[WRITE YOUR ANSWER HERE]**

# Task 2: Uniform Quantization Implementation [20%]

### Part 2a: Implement Uniform Quantization Logic [20%]

**Task:** You will now implement the core logic for uniform quantization. This involves calculating scale factors and zero-points, and then using them to map high-precision floating-point numbers to lower-precision integers.

You will use the **Straight-Through Estimator (STE)** to allow gradients to pass through the non-differentiable `round` function, which is crucial for quantization-aware training later.

#### Hint:
-   **Affine (Asymmetric):** Maps values to $[q_{\min}, q_{\max}]$.
    
-   **Symmetric:** Maps values to $[-(2^{b-1}), 2^{b-1}-1]$.
    

In [ ]:
# STE function (provided)
class _STERound(Function):
    @staticmethod
    def forward(ctx, x):
        return torch.round(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

ste_round = _STERound.apply

# Quantization configuration (provided)
@dataclass
class QuantConfig:
    num_bits: int = 8
    symmetric: bool = True
    per_channel: bool = False
    channel_axis: int = 0
    group_size: int = None

# Helper for getting min/max (provided)
def get_min_max(x, axis=None):
    if axis is None:
        return torch.min(x), torch.max(x)
    else:
        return torch.amin(x, dim=axis, keepdim=True), torch.amax(x, dim=axis, keepdim=True)

def get_scale_zero_point(x_min, x_max, cfg, qmin, qmax, device='cpu'):
    """
    Calculates the scale and zero-point for quantization.
    """
    if cfg.symmetric:
        # TODO: Implement symmetric quantization scale and zero_point
        scale = # TODO
        zero_point = # TODO
    else:
        # TODO: Implement affine (asymmetric) quantization scale and zero_point.
        scale = # TODO
        zero_point = # TODO

    # Handle the case where scale is zero to avoid division by zero errors.
    scale = torch.where(scale == 0, torch.tensor(1.0, device=device), scale)
    return scale.to(device), zero_point.to(device)

def quantize_tensor(x, scale, zero_point, qmin, qmax):
    """
    Quantizes tensor x and then dequantizes it (simulating the quantization error).
    """
    # TODO: Implement the quantization and dequantization formulas.
    # Hint: Use the ste_round function for the rounding operation.
    x_q = # TODO
    x_q_clamped = # TODO: Hint: clamp the quantized values to be within [qmin, qmax].
    x_dq = # TODO
    return x_dq

def uniform_quantize_dequantize(x: torch.Tensor, cfg: QuantConfig):
    """
    A wrapper function for uniform quantization-dequantization.
    """
    if cfg.symmetric:
        # TODO: Define qmin and qmax for symmetric quantization.
        qmin, qmax = # TODO
    else:
        # TODO: Define qmin and qmax for affine (asymmetric) quantization.
        qmin, qmax = # TODO

    # Hint: Use get_min_max, get_scale_zero_point, quantize_tensor functions
    if cfg.group_size is not None and x.ndim > 1:
        # TODO: Implement the logic for per-group quantization.

        return x_dq # dequantized tensor

    elif cfg.per_channel and x.ndim > 1:
        # TODO: Implement per-channel quantization.

        return x_dq # dequantized tensor
    else:
        # TODO: Implement per-tensor quantization.

        return x_dq # dequantized tensor


# Task 3: Non-Uniform (Power-of-Two) Quantization [10%]

### Part 3a: Implement Power-of-Two Quantization [10%]

A simple **non-uniform** scheme is **power-of-two (PoT) quantization**, where values are mapped to the nearest $\pm 2^k$. This concentrates quantization levels near zero, which often aligns well with the distribution of weights in neural networks.

**Task:** Implement the PoT quantization logic. A key challenge is handling cases where the range of exponents (`k`) exceeds the number of available quantization levels ($2^b-1$). In such cases, you must scale the exponents to fit.

In [ ]:
@dataclass
class PoTQuantConfig:
    num_bits: int = 8
    per_channel: bool = False
    channel_axis: int = 0
    eps: float = 1e-8

def pot_quantize_dequantize(x: torch.Tensor, cfg: PoTQuantConfig):
    """
    Power-of-Two (PoT) quantization and dequantization.
    Maps values to the nearest signed power-of-two.
    """
    def quantize_block(block):
        # Avoid log(0) by clamping the minimum magnitude.
        mag = block.abs().clamp(min=cfg.eps)

        # TODO: Calculate exponents
        exponents = # TODO

        kmax, kmin = exponents.max(), exponents.min()
        max_levels = (1 << cfg.num_bits) - 1
        span = (kmax - kmin + 1).clamp(min=1)

        # TODO: If the required exponent span is greater than the available levels,
        # you must scale the exponents to fit.
        if span > max_levels:
            # 1. Calculate the scaling factor.
            scale = # TODO
            # 2. Scale the exponents: e_scaled = round((e - kmin) * scale) + kmin
            exponents = # TODO

        # TODO: Reconstruct the dequantized value
        dq = # TODO

        # Ensure that original zeros remain zero.
        return torch.where(block == 0, torch.zeros_like(dq), dq)

    if cfg.per_channel and x.ndim > 1:
        # TODO: Implement per-channel PoT quantization.
        return x_dq # dequantized tensor
    else:
        return quantize_block(x)

# Task 4: Applying and Analyzing Quantization [20%]

### Part 4a: Create a Fake Quantized Linear Layer [5%]

**Task:** To apply our quantization functions to the model, we need to wrap the existing `nn.Linear` layers. You will implement `QuantLinearFake`, a module that replaces a linear layer and applies "fake" (or "simulated") quantization to its weights and activations during the forward pass. This means the computations are still done in FP32, but the values are rounded to simulate the precision loss of lower-bit representations.

In [ ]:
class QuantLinearFake(nn.Module):
    def __init__(self, linear: nn.Linear,
                 w_method="uniform", w_cfg: dict = None,
                 a_method="uniform", a_cfg: dict = None):
        super().__init__()
        self.inner = linear
        self.w_method, self.a_method = w_method, a_method
        self.w_cfg, self.a_cfg = w_cfg or {}, a_cfg or {}
        self.weight, self.bias = self.inner.weight, self.inner.bias

    def quantize_weights(self, W):
        # TODO: Implement weight quantization logic.
        # Based on self.w_method, call the appropriate quantization function
        if self.w_method == "uniform":
            return # TODO
        elif self.w_method == "pot":
            return # TODO
        else:
            return W

    def quantize_acts(self, x):
        if self.a_method == "uniform" and "num_bits" in self.a_cfg:
            cfg = QuantConfig(**self.a_cfg)
            return uniform_quantize_dequantize(x, cfg)
        else:
            return x

    def forward(self, x):
        # This forward pass is provided for you.
        xq = self.quantize_acts(x)
        Wq = self.quantize_weights(self.weight)
        return F.linear(xq, Wq, self.bias)

# This wrapper function to replace linear modules is provided for you.
def wrap_linear_modules(module: nn.Module, should_wrap_fn):
    for name, child in list(module.named_children()):
        if isinstance(child, nn.Linear):
            cfg = should_wrap_fn(name, child)
            if cfg is not None:
                wrapped = QuantLinearFake(child, **cfg)
                setattr(module, name, wrapped)
        else:
            wrap_linear_modules(child, lambda n,m: should_wrap_fn(f"{name}.{n}", m))

### Part 4b: Quantization Experiments and Analysis [15%]

**Task:** Now you will use your implemented functions to run a series of experiments (a "sweep") to analyze the impact of different quantization settings. You will compare:

1.  **Quantization Methods:** Uniform Symmetric vs. Uniform Asymmetric vs. Power-of-Two (PoT).
2.  **Bit Precision:** 16, 8, 4, and 2-bit weight quantization.
3.  **Quantization Granularity:** Per-tensor vs. per-channel vs. per-group.

The default setting is : 8-bit weight quantization, full precision activation, per-group quantization, Uniform Asymmetric

Fill in the `sweep` list below to define your experiments. Then, run the cell and analyze the resulting dataframe.

In [ ]:
# Helper functions for running the sweep (provided)
def is_opt_linear_of_interest(full_name: str):
    return any(p in ["q_proj","k_proj","v_proj","out_proj","fc1","fc2"] for p in full_name.split("."))

def make_quantized_copy(base_model, method, w_bits, a_bits=None, **kwargs):
    m = deepcopy(base_model).cpu(); m.eval()
    w_cfg = dict(num_bits=w_bits, **kwargs)
    a_cfg = dict(num_bits=a_bits, symmetric=False) if a_bits else {}
    def should_wrap(name, module):
        if is_opt_linear_of_interest(name):
            return {"w_method": method, "w_cfg": w_cfg, "a_method": "uniform", "a_cfg": a_cfg}
    wrap_linear_modules(m, should_wrap)
    return m

def eval_on_device(m, loader, device):
    m.to(device)
    loss, ppl = evaluate_perplexity(m, loader, device)
    m.to("cpu"); gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return loss, ppl

# TODO: Define your experiments in the sweep list.
# Each entry is a dictionary specifying the parameters for make_quantized_copy.
sweep = [
    # Example: {"method": "uniform", "w_bits": 8, "symmetric": False, "granularity": "per_group", "group_size": 32},

    # Task 1: Compare Quantization Methods (uniform symmetric, uniform asymmetric, power-of-two)

    # Task 2: Compare Precision (16,8,4,2 bit weight quantization)

    # Task 3: Compare Granularity (per-tensor, per-channel, per-group)

]

results = []
for params in sweep:
    gran = params.get("granularity", "per_tensor")
    params["per_channel"] = (gran == "per_channel")
    if "granularity" in params: del params["granularity"]

    qm = make_quantized_copy(model, **params)
    est_size = estimated_weight_size_bytes(qm, w_bits=params['w_bits'])
    loss, ppl = eval_on_device(qm, eval_dataloader, device)

    res_key = {**params, "granularity": gran, "loss": loss, "ppl": ppl, "size_mb": est_size / 1e6}
    results.append(res_key)

df = pd.DataFrame(results)
print("--- Experiment Results ---")
print(f"Baseline FP32 Perplexity: {ppl_fp:.3f}")
df

### Analysis of Results

Based on the dataframe above, answer the following questions.

**1. Granularity Trade-offs:** What is the relationship between quantization granularity (per-tensor, per-channel, per-group) and model perplexity? Does a finer granularity always result in better performance? What might be the hardware/efficiency trade-offs?

**[WRITE YOUR ANSWER HERE]**

**2. Quantization Methods:** Compare Uniform Symmetric, Uniform Asymmetric, and Non-uniform (PoT) quantization at 8 bits. Which performed best? Why might one be preferred over another in terms of hardware friendliness?

**[WRITE YOUR ANSWER HERE]**

**3. Precision vs. Performance:** Describe the trend you observe as you decrease the number of weight bits (from 16 down to 2). At what point does the model's performance degrade significantly?

**[WRITE YOUR ANSWER HERE]**

**4. Layer Sensitivity (Conceptual):** We don't quantize the bias, LayerNorm, embedding, or lm_head layers. Why is this a common practice? (Hint: think about parameter count, sensitivity to precision, and expected compute/memory savings).

**[WRITE YOUR ANSWER HERE]**

# Task 5: Quantization-Aware Training (QAT) [20%]

### Part 5a: Implement a QAT Training Loop [10%]

Post-Training Quantization (PTQ), which you performed in the previous task, is simple but can lead to significant accuracy degradation, especially at very low bit-widths. **Quantization-Aware Training (QAT)** is a technique to mitigate this by simulating the quantization effects during a short fine-tuning phase. This allows the model to adapt its weights to the quantization process.

**Task:** Implement a simple QAT training loop. Because we used the Straight-Through Estimator (STE) in our quantization functions, gradients can flow through the fake-quantized layers, enabling us to fine-tune the model.

In [ ]:
def qat_training(model, dataloader, device, num_epochs=3, learning_rate=1e-5):
    """
    Simple QAT training loop.
    """
    model.train() # Set the model to training mode
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0
        for batch in tqdm(dataloader, desc=f"QAT Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch.to(device)

            # TODO: Implement the training step

            optimizer.zero_grad()
            outputs = # TODO (Hint: The model computes loss when `labels` are provided).
            loss = outputs.loss
            # TODO

            total_loss += loss.item()
            num_batches += 1

        avg_loss = total_loss / num_batches
        print(f"QAT Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")

    return model

### Part 5b: Run QAT and Compare with PTQ [10%]

**Task:** We revisit the bit precision experiment from the previous task: (16, 8, 4, 2) bit weight quantization, full precision activation, uniform asymmetric, per-group quantization. Apply your QAT training loop for 3 epochs and compare the final perplexity against the PTQ (Post-Training Quantization) result for the same setting.

In [ ]:
# --- Define the quantization settings for the QAT experiment ---
# Let's try to improve a challenging 4-bit model.
QAT_PARAMS = {
    "method": "uniform",
    "w_bits": 4,
    "symmetric": False,
    "granularity": "per_group",
    "group_size": 32
}
gran = QAT_PARAMS.pop("granularity") # Handle granularity key for our function
QAT_PARAMS["per_channel"] = (gran == "per_channel")

# --- 1. Create and evaluate the PTQ model (the baseline) ---
print("--- Creating baseline PTQ model for comparison ---")
# TODO: Create the PTQ model using make_quantized_copy and QAT_PARAMS.
ptq_model = # TODO
loss_ptq, ppl_ptq = eval_on_device(ptq_model, eval_dataloader, device)

# --- 2. Create and train the QAT model ---
print(f"\n--- Starting QAT for W{QAT_PARAMS['w_bits']} {gran} model ---")
# TODO: Create the initial model for QAT using the same parameters.
qat_model = # TODO
qat_model.to(device) # Move to device for training

print("Starting QAT fine-tuning...")
qat_model = qat_training(qat_model, train_dataloader, device, num_epochs=1, learning_rate=1e-5)

# --- 3. Evaluate the final QAT model ---
print("Evaluating final QAT model...")
loss_qat, ppl_qat = eval_on_device(qat_model, eval_dataloader, device)

# --- 4. Print Final Comparison ---
print("\n--- QAT vs. PTQ Results ---")
print(f"Baseline PTQ Perplexity: {ppl_ptq:.4f}")
print(f"QAT-finetuned Perplexity: {ppl_qat:.4f}")
print("-" * 20)
print(f"Perplexity Improvement with QAT: {ppl_ptq - ppl_qat:.4f}")

### Analysis of QAT Results

**Question:** Compare the perplexity of the PTQ and QAT results across different bit-precisions (16, 8, 4, 2) bit weight quantization. Did QAT successfully recover some of the performance lost during quantization? Briefly explain why fine-tuning with simulated quantization helps the model adapt. How do PTQ and QAT perform compared to the full precision model? Do they improve? If so, why? If not, explain the performance gap.

**[WRITE YOUR ANSWER HERE]**

# Task 6: Advanced Quantization [15%]

Get familiar with the technical details of more advanced quantization method: CPT (ICLR’21)
Each answer should be 3-4 sentences long.


**1. Q1 [5%]:** What is the motivation and key insight of CPT?

**[WRITE YOUR ANSWER HERE]**

**2. Q2 [5%]:** Summarize the methodology of CPT.

**[WRITE YOUR ANSWER HERE]**

**3. Q3 [5%]:** What are the pros and cons of prior works and why is CPT better?

**[WRITE YOUR ANSWER HERE]**